Note that, in this RL game, where Deep learning really get to use, is actually gain a representation to represent the Q table. 

Since it will be hard to scale to real problem if we rely on a (State*Action) table, here we use one-hot representation (16-dim vector) to present the state, through weight matrix (directly map the state to reward) to get the reward value (which previously store in the Q table). 

In [5]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
env = gym.make('FrozenLake-v0')

[2017-02-02 09:45:36,909] Making new env: FrozenLake-v0


In [7]:
tf.reset_default_graph()

In [8]:
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [5]:
init = tf.initialize_all_variables()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False ## doomed
        j = 0 ## how many steps in one episodes take to doomed
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e: ## exploration-exploitation trade-off: less than the threshhold, then eplore
                a[0] = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
print "Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%"

[[ 253.09848022  257.14199829  266.76412964 ...,  258.97253418
   268.07540894  262.46615601]
 [ 242.88061523  252.45433044  256.07705688 ...,  251.55711365
   252.53016663  246.61167908]
 [ 253.06074524  257.13027954  263.86999512 ...,  255.32954407
   260.95227051  256.31555176]
 ..., 
 [ 248.44277954  256.68884277  261.05847168 ...,  255.19325256
   256.89810181  253.86456299]
 [ 245.3588562   251.18177795  258.14834595 ...,  250.35754395
   256.83889771  252.90406799]
 [ 244.65382385  250.92707825  260.31015015 ...,  247.97770691
   252.36830139  253.02359009]]


array([0])